# Helm & Concourse on Google Kubernetes Engine

In [2]:
!git clone https://github.com/Thekubebro/charts.git

Cloning into 'charts'...
remote: Counting objects: 32309, done.
remote: Total 32309 (delta 0), reused 0 (delta 0), pack-reused 32309
Receiving objects: 100% (32309/32309), 7.92 MiB | 23.03 MiB/s, done.
Resolving deltas: 100% (22721/22721), done.


In [3]:
cd charts

/content/charts


## Create a Kubernetes Engine cluster


In [0]:
gcloud container clusters create concourse --image-type ubuntu \
    --machine-type n1-standard-2 --zone us-central1-f \
    --scopes cloud-source-repos-ro,storage-full

## Download the sample code


In [0]:
wget https://gke-concourse.storage.googleapis.com/sample-app-v4.zip
unzip sample-app-v4.zip
cd concourse-continuous-delivery-master

## Deploying Concourse using Helm


### Install Helm

In [0]:
wget https://storage.googleapis.com/kubernetes-helm/helm-v2.6.2-linux-amd64.tar.gz

In [0]:
tar zxfv helm-v2.6.2-linux-amd64.tar.gz
cp linux-amd64/helm .

In [0]:
kubectl create clusterrolebinding user-admin-binding --clusterrole=cluster-admin --user=$(gcloud config get-value account)

In [0]:
kubectl create serviceaccount tiller --namespace kube-system

In [0]:
kubectl create clusterrolebinding tiller-admin-binding --clusterrole=cluster-admin --serviceaccount=kube-system:tiller

In [0]:
kubectl create clusterrolebinding --clusterrole=cluster-admin --serviceaccount=default:default concourse-admin

In [0]:
./helm init --service-account=tiller
./helm update

In [0]:
export PROJECT=$(gcloud info --format='value(config.project)')
export BUCKET=$PROJECT-helm-repo
./helm plugin install https://github.com/viglesiasce/helm-gcs.git --version v0.1.1
gsutil mb -l us-central1 gs://$BUCKET
./helm gcs init gs://$BUCKET

In [0]:
./helm version

In [0]:
kubectl apply -f create-helm-service.yaml

## Deploy Concourse


In [0]:
export PASSWORD=$(openssl rand -base64 15)
cat > concourse.yaml <<EOF
concourse:
  password: $PASSWORD
  baggageclaimDriver: overlay
web:
  service:
    type: LoadBalancer
EOF


In [0]:
./helm install stable/concourse --name concourse -f concourse.yaml --version 0.10.0

In [0]:
kubectl get pods -l app=concourse-web

In [0]:
export SERVICE_IP=$(kubectl get svc \
    --namespace default concourse-web \
    -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
wget -O fly "http://$SERVICE_IP:8080/api/v1/cli?arch=amd64&platform=linux"
chmod +x fly

In [0]:
./fly -t local login -u concourse -p $PASSWORD -c http://$SERVICE_IP:8080

In [0]:
export SERVICE_IP=$(kubectl get svc \
    --namespace default concourse-web \
    -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
printf "Concourse URL: [http://$SERVICE_IP:8080]\nUsername: concourse\nPassword: $PASSWORD\n"

## Configure identity and access management


In [0]:
gcloud iam service-accounts create concourse --display-name concourse

In [0]:
export SA_EMAIL=$(gcloud iam service-accounts list \
    --filter="displayName:concourse" --format='value(email)')
export PROJECT=$(gcloud info --format='value(config.project)')

In [0]:
gcloud projects add-iam-policy-binding $PROJECT \
    --role roles/storage.admin --member serviceAccount:$SA_EMAIL

In [0]:
gcloud iam service-accounts keys create concourse-sa.json \
    --iam-account $SA_EMAIL

## Deploying your application


In [0]:
gcloud source repos create chart-source
gcloud source repos create app-source

In [0]:
git config --global user.email "[EMAIL_ADDRESS]"
git config --global user.name "[USERNAME]"

In [0]:
export PROJECT=$(gcloud info --format='value(config.project)')
for repo in app-source chart-source; do
cd $repo
git init && git add . && git commit -m 'Initial commit'
git config credential.helper gcloud.sh
git remote add google \
    https://source.developers.google.com/p/$PROJECT/r/$repo
    git push --all google
cd ..
done

## Configure and create the pipeline


In [0]:
export PROJECT=$(gcloud info --format='value(config.project)')
export BUCKET=$PROJECT-helm-repo
export TOKEN_SECRET=$(kubectl get serviceaccount default -o jsonpath="{.secrets[0].name}")
export CLUSTER_CA=$(kubectl get secret $TOKEN_SECRET -o jsonpath='{.data.ca\.crt}')
export TOKEN=$(kubectl get secret $TOKEN_SECRET -o jsonpath='{.data.token}' | base64 --decode)

cat > params.yaml <<EOF
chart_name: nginx
release_name: dev-site
bucket: $BUCKET
cluster_ca: $CLUSTER_CA
token: $TOKEN
project: $PROJECT
service_account_json: '$(cat concourse-sa.json)'
EOF

In [0]:
./fly -t local set-pipeline -p dev-site-deploy \
    -c pipeline.yaml -l params.yaml -n

In [0]:
./fly -t local unpause-pipeline -p dev-site-deploy

## Deploying your application for the first time


In [0]:
for repo in app-source chart-source; do
    cd $repo
    git tag v1.0.0
    git push google --tags
    cd ..
done

In [0]:
export POD_NAME=$(kubectl get pods --namespace default \
    -l "app=nginx,release=dev-site" \
    -o jsonpath="{.items[0].metadata.name}")
kubectl port-forward $POD_NAME 8080:80 &
curl -is localhost:8080 | grep 'Server\|color'

In [0]:
killall kubectl

In [0]:
cd app-source

In [0]:
sed -i s/stable/latest/ Dockerfile

In [0]:
git add Dockerfile
git commit -m 'Use latest NGINX'
git tag v2.0.0

In [0]:
git push google --mirror

In [0]:
../fly -t local check-resource -r dev-site-deploy/app-source

## Deploying a change to the application


In [0]:
export POD_NAME=$(kubectl get pods --namespace default \
    -l "app=nginx,release=dev-site" \
    -o jsonpath="{.items[0].metadata.name}")
kubectl port-forward $POD_NAME 8080:80 &
curl -is localhost:8080 | grep 'Server\|color'

In [0]:
Server: nginx/1.15.0
<h1 style="color:blue;">Welcome to the sample app!</h1>

In [0]:
killall kubectl

## Deploying a change to the chart


In [0]:
cd ../chart-source/

In [0]:
sed -i s/blue/green/ templates/config-map.yaml

In [0]:
git add templates/config-map.yaml
git commit -m 'Use green for page heading'

In [0]:
git tag v2.0.0
git push google --mirror

In [0]:
../fly -t local check-resource -r dev-site-deploy/chart-source

In [0]:
export POD_NAME=$(kubectl get pods \
    -n default -l "app=nginx,release=dev-site" \
    -o jsonpath="{.items[0].metadata.name}")
kubectl port-forward $POD_NAME 8080:80 &
curl -is localhost:8080 | grep 'Server\|color'